In [19]:
import cv2
import time

# Load the Haar cascade file for eye detection
# This cascade is trained to detect eyes in images.
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Start capturing video from the webcam (default webcam is index 0)
cap = cv2.VideoCapture(0)

# Variable to store the start time when eyes close
eye_closed_start = None

while True:
    # Read a frame from the video capture
    ret, frame = cap.read()
    
    # If frame is not read correctly, break out of the loop
    if not ret:
        break

    # Convert the frame to grayscale for eye detection
    # Haar cascades require grayscale images.
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect eyes in the image
    eyes = eye_cascade.detectMultiScale(gray, 1.1, 4)

    # Check if no eyes are detected (indicating eyes might be closed)
    if len(eyes) == 0:
        # If eyes have just closed, record the current time
        if eye_closed_start is None:
            eye_closed_start = time.time()
        # Update the duration for which eyes are closed
        eye_closed_duration = time.time() - eye_closed_start
    else:
        # Reset the start time when eyes are open
        eye_closed_start = None

    # Draw ellipses around detected eyes
    for (x, y, w, h) in eyes:
        center = (x + w // 2, y + h // 2)  # Ellipse center
        axes = (w // 2, h // 4)  # Ellipse axes lengths
        angle = 0  # Angle of rotation of ellipse
        startAngle = 0  # Starting angle for the ellipse arc
        endAngle = 360  # Ending angle for the ellipse arc
        color = (255, 0, 0)  # Color of the ellipse (Blue)
        thickness = 2  # Thickness of the ellipse border
        cv2.ellipse(frame, center, axes, angle, startAngle, endAngle, color, thickness)

    # Display the time for which eyes have been closed on the frame
    if eye_closed_start is not None:
        # If the eyes are closed, display the ongoing duration
        cv2.putText(frame, f"Eye Closed Time: {eye_closed_duration:.2f} seconds", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    else:
        # If the eyes are open, reset and display the timer as 0.00 seconds
        cv2.putText(frame, "Eye Closed Time: 0.00 seconds", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Display the frame with the drawn ellipses and text
    cv2.imshow('Frame', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close any open windows
cap.release()
cv2.destroyAllWindows()
